In [1]:
#We need to install tf version 2.15 for compatibility with this notebook (written in March 2024).
!pip uninstall tensorflow
!pip install tensorflow==2.15.0
!tensorflow --version

Found existing installation: tensorflow 2.17.0
Uninstalling tensorflow-2.17.0:
  Would remove:
    /usr/local/bin/import_pb_to_tensorboard
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.10/dist-packages/tensorflow-2.17.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/tensorflow/*
Proceed (Y/n)? y
  Successfully uninstalled tensorflow-2.17.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.7 MB/s eta 0:00:00
  Att

# Milestone Project 1: Food Vision Big
See an annotated version of this notebook on github


## Check GPU

Google Colab offers free GPUs.  However not all are compatible with mixed precision training.  Google Colab offers:
* K80
* P100
* T4 (only one compatible with mixed precision training)

Knowing this, in order to use mixed precision training, we need access to a Tesla T4 from Google Colab or if using our own hardware, our GPU needs a score of 7.0+

In [3]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-a2239727-7ae8-ab64-5083-67e706fd1855)


## Get helper functions

In past modules we have created a bunch of helper functions to do small tasks required for our notebooks.  Rather than rewrite all of these, we can import a script and load them in from there.  The script we've got available can be found on GitHub:https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py


In [4]:
# Download helper functions script
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

--2024-11-08 04:45:50--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-11-08 04:45:51 (22.5 MB/s) - ‘helper_functions.py’ saved [10246/10246]



In [5]:
# Import series of helper functions for the notebook
from helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use TensorFlow datasets to download data

If you want to get an overview of tensorflow datasets (TFDS) read the guide

In [6]:
# Get TensorFlow datasets
import tensorflow_datasets as tfds

In [7]:
# List all of the available datasets in tensorflow
datasets_list = tfds.list_builders() # Get all the available datasets in tfds
print("food101" in datasets_list) # is our target dataset in the list of TFDS datasets?

True


In [ ]:
# Load in the data (takes 5-6 minutes in google colab)
(train_data, test_data), ds_info = tfds.load(name = "food101",
                                             split = ["train", "validation"],
                                             shuffle_files = True,
                                             as_supervised = True, # data gets returned in tuple format (data, label)
                                             with_info = True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [ ]:
# Features of Food101 from TFDS
ds_info.features

In [ ]:
# Get the class names
class_names = ds_info.features["label"].names
class_names

# Exploring the Food101 data from tenslorflow datasets
To become one with our data, we want to find:
* Class names
* The shape of our input data (image tensors)
* The datatype of our input data
* What the labels look like (ie are they one-hot encoded or label-encoded?)
* Do the labels match up with the class names?

In [ ]:
# Take one sample of the train_data
train_one_sample = train_data.take(1)   #samples are in format (image_tensor, label)

In [ ]:
# What does one sample of our training data look like?
train_one_sample

In [ ]:
# output info about our training sample
for image,label in train_one_sample:
  print(f'''
  Image shape: {image.shape},
  Image datatype: {image.dtype},
  Target class from Food101 (tensor form): {label},
  Class name (string form): {class_names[label.numpy()]}
  ''')

In [ ]:
# What does our image tensor from TFDS's Food101 look like?
image

In [ ]:
# What are the min and max values of our image tensor?
import tensorflow as tf
tf.reduce_min(image), tf.reduce_max(image)

### Plot an image from tensorflow

In [ ]:
# Plot an image tensor
import matplotlib.pyplot as plt
plt.imshow(image)
plt.title(class_names[label.numpy()]) # Add title to image to verify the label is associated with the right image
plt.axis(False)

In [ ]:
# normalize the tensor values and the image sizes



## Create preprocessing functions for our data

Neural networks perform best when data is in a certain way (eg, batched, normalized, etc.)

However, not all data, including data from tensorflow datasets, comes like this.  So in order to get it ready for a neural network, you'll often have to write preprocessing functions and map it to your data.

What we know about our data:
* It's in uint8 datatype
* It's comprised of all different sized tensors / different sized images
* Not scaled (the pixel values are between 0 and 255)

What we know models like:
* Data in `float32` dtype (or for mixed precision `float16` and `float32` dtype)
* For batches, tensorflow likes all of the tensors within a batch to be of the same size
* Scaled (Values between 0 and 1) - also called normalized tensors generally perform better

With these things in mind, we've got a few things we can tackle with a preprocessing function.

Since we're going to be using an EfficientNetBX pretrained model from tf.keras.applications, we don't need to rescale our data.  These architectures have rescaling built in.

This means our functions need to:
1. Reshape our images to all the same size
2. Convert the dtype of our image tensors from uint8 to float32


In [ ]:
# Make a function for preprocessing images
def preprocess_img(image, label, img_shape=224):
  '''
  Converts image datatype from uint8 -> float32 and reshapes image
  to [img_shape, img_shape, colour_channels]
  '''
  image=tf.image.resize(image, [img_shape, img_shape])  #reshape target image
  return tf.cast(image, tf.float32), label # return (float32_image, label) tuple

# Preprocess a single sample image and check the outputs
preprocessed_img = preprocess_img(image, label)[0]
print(f"Image before preprocessing:\n{image[:1]}..., \nShape: {image.shape}, \nDatatype: {image.dtype}\n ")
print(f"Image after processing:\n{preprocessed_img[:1]}... \nShape: {preprocessed_img.shape}, \nDatatype: {preprocessed_img.dtype}\n ")
len(image)
len(preprocessed_img)

## Batch and prepare datasets

We're now going to make our data input pipeline run really fast.

For more resources on this, i'd highly recommend going through the following guide. https://www.tensorflow.org/guide/data_performance

In [ ]:
# Map preprocessing function to training data (and parallelize)
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)

#Shuffle train_data and turn it into batches and prefetch it (load it faster)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

# Map preprocessing function to test data
test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)

In [ ]:
train_data, test_data

> "Hey TensorFlow, map this preprocessing function (`preprocess_img`) across our training dataset, then shuffle the number of elements and then batch them together and finally make sure you prepare new batches (prefetch) while the model is looking through finding patterns in the current batch"

## Create modelling callbacks

We're going to create a couple of callbacks to help us while our model trains.  Specifically:
1. Tensorboard callback (log training results so we can visualize them later if need be)
2. Model checkpoint callback (save our model's progress after feature extraction)



In [ ]:
# Create tensorboard callback (import from helper_functions.py)
from helper_functions import create_tensorboard_callback

# Create a model checkpoint callback to save a model's progress during training
checkpoint_path = "model_checkpoints/cp.ckpt"
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                      monitor="val_acc",
                                                      save_best_only=True,
                                                      verbose=0)  # don't print whether or not model is being saved

In [ ]:
tf.__version__

## Setup mixed precision training

First and foremost, for a deeper understanding of mixed precision training, check out the TensorFlow guide for mixed precision: https://www.tensorflow.org/guide/mixed_precision

Mixed precision uses a combination of float32 and float16 datatypes to speed up model performance

In [ ]:
# Turn on mixed precision training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")    #set global data policy to mixed precision

In [ ]:
mixed_precision.global_policy()

## Build feature extraction model



In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

# Create base model
input_shape = (224,224,3)
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

# Create functional model
inputs = layers.Input(shape=input_shape, name="input_layer")
# Note: efficientNetBX models have rescaling built-in, but if your model doesn't, you can have a layer like below:
# x = preprocessing.rescaling(1./255)(x)
x = base_model(inputs, training=False) # Make sure layers which should be in inference only mode stay like that
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
outputs = layers.Activation("softmax", dtype=tf.float32, name="softmax_float32")(x)
model = tf.keras.Model(inputs,outputs)

#Compile your model
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics="accuracy")



In [ ]:
model.summary()

In [ ]:
for layer in model.layers:
  print(f"name: {layer.name}, trainable: {model.trainable}, dtype: {layer.dtype}, dtype_policy: {layer.dtype_policy}")

Going through the above, we can see that:
* `Layer.name`: the human-readable name of the layer
* `layer.trainable`: is the layer trainable?
* `layer.dtype`: the data type of the stored variable
* `layer.dtype_policy`: the data type policy a layer computes on its variables with

In [ ]:
# Check the dtype policy of the efficientnetb0 layer
for layer_number, layer in enumerate(model.layers[1].layers):
  print(f"{layer_number} name: {layer.name}, trainable: {model.trainable}, dtype: {layer.dtype}, dtype_policy: {layer.dtype_policy}")

In [ ]:
# Fit the model on the training data with 3 epochs
history_101_food_classes_feature_extract = model.fit(train_data,
                                                     steps_per_epoch = len(train_data),
                                                     validation_steps = (int(0.15 * len(test_data))),
                                                      validation_data = test_data,
                                                      epochs=3,
                                                      callbacks=[model_checkpoint])

In [ ]:
# Evaluate the model on whole test data set
results_feature_extract_model = model.evaluate(test_data)
results_feature_extract_model